In [1]:
from bs4 import BeautifulSoup
import urllib
import requests
import re
import json
from urllib.parse import urljoin
from url_normalize import url_normalize #url标准化

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36',
        'Accept':'text/html,application/xhtml+xml,application/xml,image/webp,*/*;q=0.8',
        'Accept-Encoding':'gzip, deflate, br',
        'Accept-Language':'zh-CN,zh;q=0.9',
        'Connection':'keep=alive',
        'Upgrade-Insecure-Requests':'1'}

In [3]:
#抓取所有query类别小说的基本信息
rc_head = re.compile(r'(?<=<h2>).+?(?=</h2>)')
rc_url = re.compile(r'(?<=href=").+?(?=")')
rc_snippet = re.compile(r'(?<=<p>).+?(?=</p>)')
rc_src = re.compile(r'(?<=src=").+?(?=")')
def txt2soup(txt):
    return BeautifulSoup(txt)
def BaiduCQA(query='xuanhuan'):
    data = []
    root = 'https://fanqienovel.com'
    for page in range(1,11):
        url = 'https://fanqienovel.com/rank/'+query+'/page_{}'.format(page)
        try:
            txt = requests.get(url,headers=headers).content
        except Exception as e:
            print(e)
        else:
            soup = txt2soup(txt)
            html_list = list()
            for result in soup.find_all('div', class_='rank-book-item'):
                html_list.append(str(result))
            for html in html_list:
                item = dict()
                # item['html']=html
                item['url']=root+rc_url.search(html).group()
                title_txt = str(txt2soup(html).find_all('div', class_='title'))
                item['title'] = txt2soup(title_txt).text[1:-1].replace(":",",")
                author_txt = str(txt2soup(html).find_all('span', class_='font-DNMrHsV173Pd4pgy'))
                item['author'] = txt2soup(author_txt).text[1:-1]
                abstract_txt = str(txt2soup(html).find_all('div', class_='desc abstract font-DNMrHsV173Pd4pgy'))
                item['abstract'] = txt2soup(abstract_txt).text[1:-1].replace("\n",'').replace("\u3000","")
                data.append(item)
    return data,html_list
querys = ['xuanhuan','xiaozhong','dushi']
if __name__ == '__main__':
    data,_ = BaiduCQA(querys[0])
    print(data[6])
    json.dump(data,open('base_info.json','w'))

{'url': 'https://fanqienovel.com/page/6999435908046916621', 'title': '凡人问天我欲成仙', 'author': '果一桔子', 'abstract': '天地玄黄，宇宙洪荒。谁在轮回之中惊醒，又是谁在轮回之中癫狂。少年阿牛父母失踪，村寨被屠，伤心欲绝之下被带入仙门之中。杀妖，诛鬼，镇压一切邪魔。却不想这一路坎坷，天地大劫之下，他能否逃脱宿命的纠葛。且看他带着小时候得到的石罐，开启一段不寻常的修仙之路。（如果你看腻了残酷杀伐，自私自利，睚眦必报的人物形象，不妨看看另外一种修仙者的样子，此书爱恨情仇皆有，人物血肉丰满……}'}


In [4]:
import os
js1=json.load(open('base_info.json','r'))
def A_Novel(novel_id = 0,root="https://fanqienovel.com"):
    url = js1[novel_id]['url']
    title = js1[novel_id]['title']
    path = r'Novels/'+title
    if os.path.exists(path)==False:
        os.makedirs(path)
    print("爬取小说{}".format(title))
    try:
        txt = requests.get(url,headers=headers).content
    except Exception as e:
        print(e)
    else:
        soup = txt2soup(txt)
        l = list()
        for res in soup.find_all('div', class_='chapter-item'):
            result = str(res)
            chapter_url = root+rc_url.search(result).group()
            # print(txt2soup(result).text,txt2soup(result).text[1:-1])
            chapter_title = txt2soup(result).text.replace(":",",")
            novel_txt = txt2soup(requests.get(chapter_url,headers=headers).content).text
            if '下载' in novel_txt:
                break
            with open(path+'/'+'chapters_info.csv','a+',encoding='utf-8')as d:
                d.write("{},{}\n".format(chapter_title, chapter_url))
                d.close()
            with open(path+'/'+'{}.txt'.format(chapter_title),'w',encoding='utf_8') as f:
                f.write(novel_txt)
                f.close
for i in range(0,100):
    A_Novel(i)#muye-to-fanqie

爬取小说武道丹帝
爬取小说问鼎仙途
爬取小说武碎星河
爬取小说独行修仙路
爬取小说宝塔仙缘
爬取小说凡骨
爬取小说凡人问天我欲成仙
爬取小说太古神尊
爬取小说我在新手村悄悄苟成了大BOSS
爬取小说一剑霸天
爬取小说洪荒：开局夺舍了通天教主
爬取小说李大开修仙记
爬取小说寒门仙帝
爬取小说修仙是件苦差事
爬取小说凌天诛神录
爬取小说凡人飞仙
爬取小说百道战尊
爬取小说异世界：我的人生开了挂！
爬取小说五行混沌经
爬取小说偷听心声，开局云霄提剑砍定光仙
爬取小说真武剑帝
爬取小说洪荒：杨戬成圣，师傅的我苟不住
爬取小说我，被废太子，皇陵签到千年
爬取小说部族荣光
爬取小说全民御灵，我的灵宠是女僵尸
爬取小说玄幻：开局九个仙女师傅
爬取小说混沌吞天诀
爬取小说惊鸿
爬取小说剑月琴星
爬取小说青元仙府
爬取小说那些年，我们一起砍过的修仙者
爬取小说龙象金身功
爬取小说玄幻：定个小目标，成为蓝星领主
爬取小说神话复苏,开局唤醒孙悟空
爬取小说极道开天
爬取小说天珠
爬取小说玄幻：我，无敌从败家开始！
爬取小说大仙主
爬取小说洪荒：吾为第三只金乌，稳健不出
爬取小说凡人飞升录
爬取小说神算：开局拒绝女帝！
爬取小说冒牌御兽师
爬取小说天下游鱼
爬取小说得道升仙
爬取小说玄幻：我能无限加点
爬取小说凡人修仙之随身空间
爬取小说石破天奇
爬取小说剑问天地
爬取小说反派：我的弟弟是天选之子
爬取小说我的师兄太强了
爬取小说绝代剑神
爬取小说损道友莫损贫道
爬取小说修仙家族之青玉仙葫
爬取小说从死囚营杀出的极道武圣
爬取小说大荒九龙诀
爬取小说仙海蜉蝣
爬取小说家族修仙，气运签到六十年
爬取小说剑道擎天
爬取小说通仙灵图
爬取小说签到成圣：发现老婆竟是魔道女帝
爬取小说异世召唤千古人杰
爬取小说人在娘胎：把女帝气成早产儿
爬取小说修仙
爬取小说灵眼仙缘
爬取小说洪荒：证道大罗了金手指才来
爬取小说修真枪械师
爬取小说剑荡乾坤
爬取小说玄幻：我率领亿万阴兵，重建地府
爬取小说抽除杂质，臭虫变真龙
爬取小说玄幻：诸天最强宗门
爬取小说神宠进化系统秒速升级
爬取小说以武证道：我！人族大都督
爬取小说无道仙路
爬取小说断武心痕
爬取小说不灭神心
爬取小说西游：从人间武圣开始
爬取小说神宠时代：我有一个龙族编辑器
爬取小说洪荒：我苟在元凤腹中，稳健不出
爬取小说太古神主
爬取小说圣神重生：从当

In [5]:
def get_stopwords():
    stopwords=[]
    text=open(r"D:\桌面\大三上\智能信息检索导论\code\hw2\中文stopword.txt","rt",encoding='utf-8')
    f=text.read()
    f=f.replace("\n",",")
    f=f.replace(" ",',')
    stopwords.append(f.split(","))
    stopwords=stopwords[0]
    stopwords1=['the','a','in','of','on']
    for i in range(len(stopwords1)):
        stopwords.append(stopwords1[i])
    return stopwords
def filter_word(word,stopwords):
    a=1
    if len(word)<=1:
        a=0
    elif word in stopwords:
        a=0
    return a
stopwords = get_stopwords()

In [5]:
from collections import Counter
import numpy as np
import os
import jieba
import pickle
def abstract_jieba(data):
    term_docid_pairs = []
    abstract_length = []
    all_terms=set()#建立所有词的集合，之后做一个idf的字典方便查询计算tf—idf
    N = len(data)
    for i in range(N):
        terms = [term for term in jieba.lcut_for_search(data[i]['abstract'].replace(".",'').replace("-",'')) if filter_word(term,stopwords) == 1 and len(term)>1]
        # 构造term_docid_pairs
        for term in terms:
            term_docid_pairs.append((term, i))
            all_terms.add(term)
        # 统计tf_ 
        term_counts = np.array(list(Counter(terms).values()))
        log_tf = np.log(1 + term_counts) 
        # log_tf = 1+np.log(term_counts) 
        abstract_length.append(np.sqrt(np.sum(log_tf**2))) #每个向量的长度,不在这考虑idf的话，每个向量少乘以一个固定值，都一样
    return term_docid_pairs,abstract_length,all_terms
# term_docid_pairs,abstract_length,all_terms = abstract_jieba(js1)
# print(term_docid_pairs[:5])
# print('------------------------------------------------------')
# print(term_docid_pairs[-5:])
# print(abstract_length[:5])

In [4]:
from collections import defaultdict
# 构造倒排索引
# postings list中每一项为一个Posting类对象
class Posting(object):
    def __init__(self, docid, tf=0):
        self.docid = docid
        self.tf = tf
    def __repr__(self):
        return "<docid: %d, tf: %d>" % (self.docid, self.tf)
def abstract_jieba(term_docid_pairs):
    # 排序
    # 默认情况下两个tuple比较大小就是先按照第一个元素比较，再按照第二个元素比较，因此不需要用key=lambda x:...的方式定义排序方式。
    term_docid_pairs = sorted(term_docid_pairs)
    inverted_index = defaultdict(lambda: [Posting(-1, 0)])  
    for term, docid in term_docid_pairs:
        postings_list = inverted_index[term]
        if docid != postings_list[-1].docid:
            postings_list.append(Posting(docid, 1))
        else:
            postings_list[-1].tf += 1
    inverted_index = dict(inverted_index)
    return inverted_index
# inverted_index = abstract_jieba(term_docid_pairs)

In [8]:
inverted_index['天之骄子']

[<docid: -1, tf: 0>, <docid: 55, tf: 1>]

In [9]:
def frequency_making(all_terms_,inverted_index_,N):#做一个idf列表方便查询
    frequency=[]
    word=[]
    for term in all_terms_:
        word.append(term)
        frequency.append(N/(len(inverted_index_[term])-1))#-1是排除之前添加的（-1，x）的posting
    fre=np.array(frequency)
    fre=np.log(fre)
    frequency_dict_=dict(zip(word,fre))
    return frequency_dict_
N = len(js1)
frequency_dict=frequency_making(all_terms,inverted_index,N)#文本的idf
frequency_dict['天之骄子']

4.605170185988092

In [6]:
data={"frequency_dict":frequency_dict,
              "abstract_length":abstract_length,
              "inverted_index":inverted_index,
              'data':js1
              }
with open("abstract.pickle",'wb') as f:
    pickle.dump(data,f,pickle.HIGHEST_PROTOCOL)

NameError: name 'js1' is not defined

In [5]:
import pickle
with open(r"abstract.pickle",'rb') as f:
    data=pickle.load(f),
frequency_dict = data[0]['frequency_dict']
abstract_length = data[0]['abstract_length']
inverted_index = data[0]['inverted_index']
data1 = data[0]['data']

In [6]:
def get_postings_list(inverted_index, query_term):
    try:
        return inverted_index[query_term][1:]
    except KeyError:
        return []
def cosine_scores_TAAT(inverted_index, doc_length, query, k=3):
    scores = defaultdict(lambda: 0.0) # 保存分数
    query_terms = Counter(term for term in jieba.lcut_for_search(query)) # 对查询进行分词
    #print(query_terms)_Counter({'...': 1})
    query_len=[]
    for q in query_terms:
        if q in frequency_dict:
            idf=np.array(frequency_dict[q])
        else:
            continue
        w_tq = np.log(1 + query_terms[q])*idf
        #w_tq = (1+np.log( query_terms[q]))*idf
        query_len.append(w_tq)
        postings_list = get_postings_list(inverted_index, q)
        for posting in postings_list:
            w_td = np.log(1 + posting.tf)
            scores[posting.docid] += w_td * w_tq
    zc=np.array(query_len)
    lenth_=np.sqrt(np.sum(zc**2))#考虑idf后对向量的归一化
    results = [(score /(( doc_length[docid]+1)*lenth_), docid) for docid, score in scores.items()]
    sorted(results)[::-1]
    return results[0:k]
cosine_scores_TAAT(inverted_index, abstract_length, query='天才',k=10)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\姚鑫浩\AppData\Local\Temp\jieba.cache
Loading model cost 1.258 seconds.
Prefix dict has been built successfully.


[(0.09122749333351582, 35),
 (0.16693250227314974, 49),
 (0.10778233415861661, 67),
 (0.16596764591023644, 73)]

In [8]:
def retrieval_by_cosine_sim(inverted_index,abstract_length, data, query, k=20):
    top_scores = cosine_scores_TAAT(inverted_index, abstract_length, query, k=k)
    results = [ {'url':data[docid]['url'],'title':data[docid]['title'],'author':data[docid]['author'],'abstract':data[docid]['abstract']} for score,docid in top_scores]
    return results
def evaluate(query:str) -> list:
    url_list=retrieval_by_cosine_sim(inverted_index,abstract_length, data1, query=query, k=10)
    return url_list
evaluate('废物')

[{'url': 'https://fanqienovel.com/page/6792789104695184392',
  'title': '太古神尊',
  'author': '楚长歌',
  'abstract': '【热血玄幻+霸气铁血+杀伐果断+暴爽节奏+吞噬突破+天赋觉醒+体质进化+剑道】武道修行世界，辉煌的造化神朝因麾下宰相的背叛，皇权覆灭，神帝被囚禁，废物太子被杀，三千年后，太子叶风重生归来，这一世觉醒绝世天赋，铸造强大神体，诛杀强敌，鏖战天下，向三千年后的新一代神朝发起进攻！“待我重生归来时，一剑杀尽天下敌！”'},
 {'url': 'https://fanqienovel.com/page/6979754580720487459',
  'title': '全民御灵，我的灵宠是女僵尸',
  'author': '七七椰奶',
  'abstract': '叶凌意外重生，来到了一个神奇的世界，一个全民御灵的时代。成功激活御灵系统，可以看到常人所不知道的隐藏进化方向。每提升一阶，就可以契约一个新的灵宠。而叶凌的第一个灵宠，竟然是僵尸，一个在世人眼中，废物得不行的僵尸。但是在觉醒系统的叶凌眼中，自己的僵尸却是一个超级灵宠！他的僵尸，不仅战力十足，而且可颜可飒！不仅是第一个灵宠，之后叶凌的每一个灵宠，全都能进化为人形。'},
 {'url': 'https://fanqienovel.com/page/6949745138713234446',
  'title': '玄幻：我率领亿万阴兵，重建地府',
  'author': '隔壁小江',
  'abstract': '陈七夜穿越到玄幻世界，发现周围有无数厉鬼横行，害人索命。幸好，体内觉醒了最强阴天子系统。召唤黑白无常，拘拿亡魂，斩杀恶鬼！召唤阴兵阴将，鬼门大开，百鬼夜行！召唤判官城隍，惩恶扬善，坐镇世间！召唤酆都鬼城，收容万鬼，界定轮回！“什么？你是萧家废物，逆天崛起？拘！”“什么？你是气运之子，有大帝之姿？拘！”管你妖孽天才还是盖世强者，只要进了我地府，是龙你给我盘着，是虎你给我趴着！'}]